In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import re

In [3]:
df=pd.read_csv('resume_dataset.csv')
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [5]:
df['Category'].value_counts()

Category
Java Developer               84
Testing                      70
DevOps Engineer              55
Python Developer             48
Web Designing                45
HR                           44
Hadoop                       42
Blockchain                   40
ETL Developer                40
Operations Manager           40
Data Science                 40
Sales                        40
Mechanical Engineer          40
Arts                         36
Database                     33
Electrical Engineering       30
Health and fitness           30
PMO                          30
Business Analyst             28
DotNet Developer             28
Automation Testing           26
Network Security Engineer    25
SAP Developer                24
Civil Engineer               24
Advocate                     20
Name: count, dtype: int64

In [7]:
df['Resume'][0]

'Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details \r\n\r\nData Science Assurance Associate \r\n\r\nData Science Assurance Associate - Ernst & Young LLP\r\nSkill Details \r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nPython- Exprience - 24 monthsCompany Details \r\ncompany - Ernst & Young LLP\r\ndescription - Fraud Investigatio

In [9]:
import re

def cleanResume(txt):
    cleanTxt = re.sub(r'http\S+', ' ', txt)             # remove URLs
    cleanTxt = re.sub(r'\b(RT|cc)\b', '', cleanTxt)     # remove RT or cc only as words
    cleanTxt = re.sub(r'[@#]', ' ', cleanTxt)           # remove @ and #
    cleanTxt = re.sub(r'[^\x00-\x7f]', ' ', cleanTxt)   # remove non-ASCII
    cleanTxt = re.sub(r'[^a-zA-Z ]', ' ', cleanTxt)     # keep only letters + spaces
    cleanTxt = re.sub(r'\s+', ' ', cleanTxt)            # collapse multiple spaces
    return cleanTxt.strip()

In [11]:
cleanResume("my ### $ # >ram websiite like is access this http://helloworld@gmail ^\\x 00 cc ")

'my ram websiite like is access this x'

In [13]:
df['Resume']=df['Resume'].apply(lambda x:cleanResume(x))

In [15]:
df['Resume'][5]

'SKILLS C Basics IOT Python MATLAB Data Science Machine Learning HTML Microsoft Word Microsoft Excel Microsoft Powerpoint RECOGNITION Academic Secured First place in B Tech Education Details August to May B Tech Ghatkesar Andhra Pradesh Aurora s Scientific and Technological Institute June to May Secondary Education Warangal Telangana SR Junior College Data Science Skill Details MS OFFICE Exprience Less than year months C Exprience Less than year months machine learning Exprience Less than year months data science Exprience Less than year months Matlab Exprience Less than year monthsCompany Details company description'

In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [19]:
df['Category']=le.fit_transform(df['Category'])

In [21]:
df['Category'].unique()

array([ 6, 12,  0,  1, 24, 16, 22, 14,  5, 15,  4, 21,  2, 11, 18, 20,  8,
       17, 19,  7, 13, 10,  9,  3, 23])

**CONVERTING TEXT INTO NUMBER OF RESUME COLUMN**

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(stop_words='english')


tfidf.fit(df['Resume'])
requiredTxt=tfidf.transform(df['Resume'])
requiredTxt

<962x7036 sparse matrix of type '<class 'numpy.float64'>'
	with 158292 stored elements in Compressed Sparse Row format>

In [26]:
from sklearn.model_selection import train_test_split

In [28]:
X=requiredTxt
y=df['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**MODEL TRAINING(IMPORTANT)**

**1st Model Training Method**

**KNN with OneVsRestClassifier** Use this when your problem is single-label classification

In [32]:
# Model training using KNN(Classification) and OnevsRestClassifier


from sklearn.neighbors import KNeighborsClassifier #new
from sklearn.multiclass import OneVsRestClassifier #new
from sklearn.metrics import accuracy_score

clf=OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train,y_train)
ypred=clf.predict(X_test)
print(accuracy_score(y_test,ypred))

0.9844559585492227


**Plain KNN (default scikit-learn)**   Use this when your problem is multi-label classification

In [34]:
# Model training using KNN(Classification)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Initialize model
knn = KNeighborsClassifier(n_neighbors=5)

# Train
knn.fit(X_train, y_train)

# Predict
y_pred = knn.predict(X_test)

# Accuracy
print("Plain KNN Accuracy:", accuracy_score(y_test, y_pred)*100)


Plain KNN Accuracy: 98.44559585492227


**Second Model Training Method**

In [41]:
# RANDOM FOREST CLASSIFICATION

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred)*100)


Random Forest Accuracy: 99.48186528497409


**SAMPLE RESUME TEXT**

In [44]:
sample_resume = """
Experienced in Python, Machine Learning, Data Analysis, Pandas, Numpy, and Deep Learning.
Worked on projects involving classification, clustering, and NLP http://gmail.com $hii.
"""
sample= tfidf.transform([sample_resume])
predicted_role = clf.predict(sample)[0]
print("Predicted Job Role:", predicted_role)

Predicted Job Role: 6


In [46]:
import pickle
pickle.dump(tfidf,open('tfidf.pkl','wb'))
pickle.dump(clf,open('clf.pkl','wb'))

In [48]:
import pickle

clf=pickle.load(open('clf.pkl','rb'))

cleaned_Resume=cleanResume(sample_resume)

input_features=tfidf.transform([cleaned_Resume])

prediction_id=clf.predict(input_features)[0]

categoryMapping={
    15:'Java Developer',
    23:'Testing',
    8:'DevOps Engineer',
    20:'Python Developer',
    24:'Web Designing',
    12:'HR',
    13:'Hadoop',
    3:'Blockchain',
    10:'ETL Developer',
    18:'Operations Manager',
    6:'Data Science',
    22:'Sales',
    16:'Mechanical Engineer',
    1:'Arts',
    7:'Database',
    11:'Electric Engineering',
    14:'Health and fitness',
    19:'PMO',
    4:'Business Analyst',
    9:'Dotnet Developer',
    2:'Automation Testing',
    17:'Network Security Engineer',
    21:'SAP Developer',
    5:'Civil Engineer',
    0:'Advocate'
}

category_name=categoryMapping.get(prediction_id,'Unknown')

print("Predicted Category:",category_name)

Predicted Category: Data Science
